# Data classification

# Features Selection with SHAP

## Step 1: Train an XGBoost model with all features

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import shap
import matplotlib.pyplot as plt

def load_data(file_path, feature_cols, label_cols):
    """Load dataset and split into features and labels."""
    try:
        df = pd.read_csv(file_path)
        print(f"Loaded data from {file_path}, shape: {df.shape}")
        X = df[feature_cols]
        y = df[label_cols]
        return X, y, df
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return None, None, None
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return None, None, None

def train_model(X_train, y_train):
    """Train a MultiOutputClassifier with XGBoost."""
    base_model = xgb.XGBClassifier(
        eval_metric='logloss',
        enable_categorical=False
    )
    multi_model = MultiOutputClassifier(base_model)
    multi_model.fit(X_train, y_train)
    print("Model training completed.")
    return multi_model

def compute_shap_values(multi_model, X_test, label_cols):
    """Compute SHAP values for each label and calculate mean global importance."""
    shap_matrix = pd.DataFrame(index=X_test.columns)

    for i, target in enumerate(label_cols):
        print(f"Processing condition: {target}")
        single_model = multi_model.estimators_[i]
        explainer = shap.Explainer(single_model)
        shap_values = explainer(X_test)
        mean_shap = np.abs(shap_values.values).mean(axis=0)
        shap_matrix[target] = mean_shap

    shap_matrix['mean_global'] = shap_matrix.mean(axis=1)
    return shap_matrix

def visualize_top_features(shap_matrix, n=20):
    """Visualize top N features by global SHAP importance."""
    shap_ranked = shap_matrix.sort_values('mean_global', ascending=False)
    top_features = shap_ranked.head(n)

    plt.figure(figsize=(10, 6))
    top_features['mean_global'].plot(kind='barh')
    plt.gca().invert_yaxis()
    plt.title(f"Top {n} Features (Global SHAP Importance)")
    plt.xlabel("Mean(|SHAP|) across all labels")
    plt.tight_layout()
    plt.show()

    display(top_features)
    return top_features

def evaluate_thresholds(X_train, X_test, y_train, y_test, shap_matrix, thresholds, label_cols):
    """Evaluate model performance across different SHAP thresholds."""
    results = []

    for threshold in thresholds:
        selected_features = shap_matrix[shap_matrix['mean_global'] >= threshold].index.tolist()

        if not selected_features:
            print(f"🔴 Threshold {threshold:.3f}: no features selected.")
            continue

        X_train_sel = X_train[selected_features]
        X_test_sel = X_test[selected_features]

        model_sel = MultiOutputClassifier(xgb.XGBClassifier(
            eval_metric='logloss',
            enable_categorical=False
        ))
        model_sel.fit(X_train_sel, y_train)

        y_pred = model_sel.predict(X_test_sel)

        # Multi-label evaluation
        f1 = np.mean([f1_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0, average='macro') for i in range(y_test.shape[1])])
        precision = np.mean([precision_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0) for i in range(y_test.shape[1])])
        recall = np.mean([recall_score(y_test.iloc[:, i], y_pred[:, i], zero_division=0) for i in range(y_test.shape[1])])

        try:
            y_prob = np.column_stack([model_sel.estimators_[i].predict_proba(X_test_sel)[:, 1] for i in range(y_test.shape[1])])
            auc = np.mean([roc_auc_score(y_test.iloc[:, i], y_prob[:, i]) for i in range(y_test.shape[1])])
        except Exception as e:
            print(f"Warning: AUC calculation failed for threshold {threshold:.3f}: {str(e)}")
            auc = np.nan

        print(f"✅ Threshold {threshold:.3f} | {len(selected_features)} features | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUC: {auc:.4f}")

        results.append({
            'threshold': threshold,
            'num_features': len(selected_features),
            'f1_macro': f1,
            'precision_macro': precision,
            'recall_macro': recall,
            'roc_auc_macro': auc
        })

    results_df = pd.DataFrame(results)
    display(results_df.sort_values(by='f1_macro', ascending=False))
    return results_df

def save_selected_features_dataset(df, selected_features, feature_cols, label_cols, output_path):
    """Save dataset with selected features, metadata, and labels."""
    if not selected_features:
        print("No features selected for the given threshold.")
        return

    final_selected_df = df[selected_features]
    metadata_columns = ['cow', 'duration_hours']
    final_selected_df_1 = pd.concat([
        df[metadata_columns],
        final_selected_df,
        df[label_cols]
    ], axis=1)

    try:
        final_selected_df_1.to_csv(output_path, index=False)
        print(f"Selected features dataset saved to '{output_path}'.")
        print(final_selected_df_1.head())
        print(f"Columns in saved dataset: {list(final_selected_df_1.columns)}")
    except Exception as e:
        print(f"Error saving to {output_path}: {str(e)}")

# Define feature and label columns
feature_cols = [
    'Minimum', 'Maximum', 'Mean', 'RMS', 'STD', 'MeanSTD6h', 'STDMean6h', 'STDSD',
    'RMSSD', 'Mode', 'Q10', 'Q90', 'Q25', 'Q50', 'Q75',
    'Skewness', 'Kurtosis'
] + [f'Autocorr{i}' for i in range(1, 12)] + [f'h{i}' for i in range(1, 5)]
label_cols = ['oestrus', 'calving', 'lameness', 'mastitis', 'other_disease', 'OK']

# Main execution
X, y, final_df = load_data('balanced_dataset_all.csv', feature_cols, label_cols)
if X is not None and y is not None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    multi_model = train_model(X_train, y_train)
    shap_matrix = compute_shap_values(multi_model, X_test, label_cols)
    top_features = visualize_top_features(shap_matrix, n=20)

    thresholds = np.arange(0.05, 0.3, 0.02)
    results_df = evaluate_thresholds(X_train, X_test, y_train, y_test, shap_matrix, thresholds, label_cols)

    # Select features with optimal threshold (based on original notebook comment)
    threshold = 0.15
    selected_features = shap_matrix[shap_matrix['mean_global'] >= threshold].index.tolist()
    print(f"🎯 Selected threshold = {threshold}")
    print(f"✅ {len(selected_features)} features selected:\n{selected_features}")

    save_selected_features_dataset(final_df, selected_features, feature_cols, label_cols, 'selected_features_after_augmentation.csv')
